In [1]:
#Second test for git aliases (e.g., git-update-plus, git-next-checkin, etc)

# Note: The aliases should be defined before Jupyter is invoked.
# For BatsPP, they can loaded via '--source all-tomohara-aliases-etc.bash' option.

In [2]:
# Global Setup
# Creating temp local repos to test on

TMP=/tmp/temp-git-test

In [3]:
# Moved as Global Setup cannot produce output
git clone https://github.com/LorenzoFerraces/bash-test.git "$TMP"

# Move to second local to push some changes
command cd "$TMP" 
git pull origin main 
echo "hello second local" > hello.txt
git add hello.txt
git commit -m "first commit on second local"
git push


fatal: destination path '/tmp/temp-git-test' already exists and is not an empty directory.
From https://github.com/LorenzoFerraces/bash-test
 * branch            main       -> FETCH_HEAD
Already up to date.
bash: hello.txt: cannot overwrite existing file
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
remote: Permission to LorenzoFerraces/bash-test.git denied to aveeyan.
fatal: unable to access 'https://github.com/LorenzoFerraces/bash-test.git/': The requested URL returned error: 403


: 128

In [ ]:
# git-rename-file tests
# Assert file doesn't exist
[ ! -f "newHello.txt" ]; echo $?;

In [ ]:
export GIT_MESSAGE="renamed hello.txt"
previous=$GIT_NO_CONFIRM
export GIT_NO_CONFIRM="1"
contents="$(cat hello.txt)"
git-rename-file hello.txt newHello.txt

In [ ]:
# Asserts
[ -f "newHello.txt" ] && [ "$(cat newHello.txt)" == "$(echo $contents)" ]; 
echo $?;

In [ ]:
# git-restore-staged-alias test
echo "second change on local" >> newHello.txt
git add newHello.txt

In [ ]:
# Assert theres is one staged file 
[ $(git status -s | grep -c "^[MTADRCU]") -eq 1 ]; echo $?; 

In [ ]:
export GIT_MESSAGE="restored newHello.txt"
export GIT_NO_CONFIRM="1"
git-restore-staged-alias newHello.txt
rm -rf _git-trash

In [ ]:
# Assert theres are no staged files
[ $(git status -s | grep -c "^[MTADRCU]") -eq 0 ]; echo $?; 

In [ ]:
# git-revert-file-alias tests
echo "second change on local" >> newHello.txt


In [ ]:
# Assert theres is 1 unstaged file
[ $(git status -s | grep -c "^\s[MTADRCU]") -eq 1 ]; echo $?; 

In [ ]:
#revert unstaged file
export GIT_MESSAGE="reverted newHello.txt"
export GIT_NO_CONFIRM="1"
git-revert-file-alias newHello.txt
rm -rf _git-trash

In [ ]:
# Assert file was reverted
# this matches any files except untracked
# read https://git-scm.com/docs/git-status#_short_formatz
[ $(git status -s | grep -c "^\s*[MTADRCU]") -eq 0 ]; echo $?; 

In [ ]:
# git-move-to-dir tests
mkdir folder
git add folder
git-move-to-dir "$TMP/folder" newHello.txt | true
command cd folder

In [ ]:
#Assert the file has been moved to folder
[ $(ls | wc -l) -eq 1 ]; echo $?;


In [ ]:
# git-conflicts-alias tests
command cd $TMP

# first assert there are no conflicts
[ $(git-conflicts-alias | wc -l) -eq 0 ]; echo $?

In [ ]:
# create branches that conflict later
git branch branch-a
git branch branch-b

git checkout branch-a
echo "line a" >> conflicts.txt
git add . 
git commit -m "first commit branch-a"

git switch branch-b
echo "line b" >> conflicts.txt
git add . 
git commit -m "first commit branch-b"

In [ ]:
# merge branches 
git switch main
git merge branch-a
git merge branch-b | true

In [ ]:
# assert there are now conflicts 
[ $(git-conflicts-alias | wc -l) -gt 0 ]; echo $?;

In [ ]:
# undo changes and remove newly created files

# TODO2:
## also, converting unique ID's to codes like temp-git-test/.git/objects/pack/pack-HEX.idx
## EX: cat $log | perl -pe 's/\b[0-9]+\b/NUM/g; s/\b[a-f0-9]+\b/HEX/g;'
rm -f *.txt > /dev/null
git rm -f *.txt > /dev/null
git rm -rf folder
git add . > /dev/null
git commit -m "cleanup test files"
git push

In [ ]:
# delete test local repos

command cd "/tmp"
rm -rf temp-git-test > /dev/null
export GIT_NO_CONFIRM=$previous
